In [ ]:
!python -m pip install --upgrade pip

In [ ]:
# image name preferably should start with sagemaker
!docker build -t sagemaker_feature_encoder8 -f Dockerfile .

In [ ]:
%%sh

# Specify a name to your custom container
container_name=sagemaker_feature_encoder8
echo "Container Name: " ${container_name}

# Retreive AWS account ID
account=$(aws sts get-caller-identity --query Account --output text)

# Get the AWS region defined in the current configuration (default to us-east-1 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

echo "Account: " ${account}
echo "Region: "${region}

repository="${account}.dkr.ecr.${region}.amazonaws.com"
echo "ECR Repository: " ${repository}

image="${account}.dkr.ecr.${region}.amazonaws.com/${container_name}:latest"
echo "ECR Image URI: " ${image}

# If the ECR repository does not exist, create it.
aws ecr describe-repositories --repository-names ${container_name} > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name ${container_name} > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${repository}

# Tag the local image with ECR image name
docker tag ${container_name} ${image}

# Finally, push the local docker image to ECR with the full ECR image name
docker push ${image}

In [ ]:
#!docker rmi $(docker images -q) -f

## Deploy feature encoder as a SageMaker real time endpoint

### Imports 

In [ ]:
from time import gmtime, strftime
import sagemaker
import datetime
import boto3

In [ ]:
role = sagemaker.get_execution_role()
session = sagemaker.Session()
account = session.boto_session.client('sts').get_caller_identity()['Account']
region = session.boto_session.region_name
sagemaker_client = boto3.client('sagemaker', region_name=region)
image_name = 'sagemaker_feature_encoder8'
image_uri = f'{account}.dkr.ecr.{region}.amazonaws.com/{image_name}:latest'
image_uri

In [ ]:
current_timestamp = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_name = f'feature-encoder-{current_timestamp}'

In [ ]:
!tar -czf encoder.tar.gz -C ./data .

In [ ]:
default_bucket = session.default_bucket()
default_bucket

In [ ]:
!aws s3 cp encoder.tar.gz s3://{default_bucket}/

In [ ]:
model_artifacts_location = f's3://{default_bucket}/encoder.tar.gz'

In [ ]:
response = sagemaker_client.create_model(ModelName=model_name, 
                                         Containers=[{
                                             "Image": image_uri, 
                                             "Mode": "SingleModel", 
                                             "ModelDataUrl": model_artifacts_location}], 
                                         ExecutionRoleArn=role)

print(f'Model Arn: {response["ModelArn"]}')

In [ ]:
ep_config_name = "sm-feature-encoder" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=ep_config_name,
    
    
    ProductionVariants=[
        {
            "VariantName": "v1",
            "ModelName": model_name,
            "InstanceType": "ml.m5.xlarge",
            'InitialInstanceCount': 2,
                        'InitialVariantWeight': 1,}
    ]
)

print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

In [ ]:
response = sagemaker_client.create_endpoint(
              EndpointName       = ep_config_name + '1',
              EndpointConfigName = ep_config_name)
response

In [ ]:


sagemaker_runtime = boto3.client("sagemaker-runtime", region_name='us-east-1')



In [ ]:
%%time

endpoint_name = ep_config_name + '1'
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=b"10-19-2021,145,24,22,14,usa,65+",
    ContentType="text/csv",
)
response

In [ ]:


predicted_label = response['Body'].read().decode('utf-8').strip()
predicted_label

